<a href="https://colab.research.google.com/github/MrudulMascarenhas/INS_TASK/blob/main/hybrid1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import math

In [2]:
def mod_inv(a, m):
    def egcd(a, b):
        if a == 0:
            return (b, 0, 1)
        else:
            g, y, x = egcd(b % a, a)
            return (g, x - (b // a) * y, y)
    g, x, _ = egcd(a, m)
    if g != 1:
        raise Exception("Modular inverse does not exist")
    return x % m

In [3]:
def matrix_mod_inv(matrix, m):
    n = matrix.shape[0]
    det = int(round(np.linalg.det(matrix)))
    det_inv = mod_inv(det % m, m)
    adj = np.zeros((n, n), dtype=int)
    for i in range(n):
        for j in range(n):
            minor = np.delete(np.delete(matrix, i, axis=0), j, axis=1)
            cofactor = ((-1) ** (i + j)) * int(round(np.linalg.det(minor)))
            adj[j][i] = cofactor % m
    return (det_inv * adj) % m

In [4]:
def hill_encrypt(plaintext, key_matrix, m=26):
    """Encrypt plaintext using the Hill cipher."""
    plaintext = plaintext.upper().replace(" ", "")
    n = key_matrix.shape[0]
    while len(plaintext) % n != 0:
        plaintext += "X"
    ciphertext = ""
    for i in range(0, len(plaintext), n):
        block = plaintext[i:i+n]
        vector = np.array([ord(c) - ord('A') for c in block]).reshape(n, 1)
        encrypted_vector = np.dot(key_matrix, vector) % m
        block_cipher = "".join(chr(int(num) + ord('A')) for num in encrypted_vector)
        ciphertext += block_cipher
    return ciphertext

In [5]:
def hill_decrypt(ciphertext, key_matrix, m=26):
    n = key_matrix.shape[0]
    inv_matrix = matrix_mod_inv(key_matrix, m)
    plaintext = ""
    for i in range(0, len(ciphertext), n):
        block = ciphertext[i:i+n]
        vector = np.array([ord(c) - ord('A') for c in block]).reshape(n, 1)
        decrypted_vector = np.dot(inv_matrix, vector) % m
        block_plain = "".join(chr(int(num) + ord('A')) for num in decrypted_vector)
        plaintext += block_plain
    return plaintext

In [6]:
def columnar_transposition_encrypt(text, key):
    num_cols = len(key)
    num_rows = math.ceil(len(text) / num_cols)
    padded_text = text + "X" * (num_rows * num_cols - len(text))
    grid = [padded_text[i * num_cols:(i + 1) * num_cols] for i in range(num_rows)]
    key_order = sorted(list(enumerate(key)), key=lambda x: x[1])
    ciphertext = ""
    for idx, _ in key_order:
        for row in grid:
            ciphertext += row[idx]
    return ciphertext

In [7]:
def columnar_transposition_decrypt(ciphertext, key):
    """Decrypt ciphertext using the Columnar Transposition cipher."""
    num_cols = len(key)
    num_rows = math.ceil(len(ciphertext) / num_cols)
    key_order = sorted(list(enumerate(key)), key=lambda x: x[1])
    grid = [[''] * num_cols for _ in range(num_rows)]
    k = 0
    for idx, _ in key_order:
        for i in range(num_rows):
            grid[i][idx] = ciphertext[k]
            k += 1
    plaintext = "".join("".join(row) for row in grid)
    return plaintext.rstrip("X")

In [8]:
def hybrid_encrypt(plaintext, hill_key_matrix, transposition_key):
    hill_ciphertext = hill_encrypt(plaintext, hill_key_matrix)
    final_ciphertext = columnar_transposition_encrypt(hill_ciphertext, transposition_key)
    return final_ciphertext

In [9]:
def hybrid_decrypt(ciphertext, hill_key_matrix, transposition_key):
    hill_ciphertext = columnar_transposition_decrypt(ciphertext, transposition_key)
    plaintext = hill_decrypt(hill_ciphertext, hill_key_matrix)
    return plaintext


In [10]:
if __name__ == "__main__":
    plaintext = "HELLO WORLD"
    hill_key = np.array([[3, 3],
                         [2, 5]])
    transposition_key = "3142"

    encrypted = hybrid_encrypt(plaintext, hill_key, transposition_key)
    print("Encrypted:", encrypted)

    decrypted = hybrid_decrypt(encrypted, hill_key, transposition_key)
    print("Decrypted:", decrypted)

Encrypted: IILZJXHEQOPX
Decrypted: HELLOWORLD


<ipython-input-4-811bf707e4fa>:12: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  block_cipher = "".join(chr(int(num) + ord('A')) for num in encrypted_vector)
<ipython-input-5-7c5909190abe>:9: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  block_plain = "".join(chr(int(num) + ord('A')) for num in decrypted_vector)
